# __Test another way to finetune BERT__

Based on [Classify text with BERT](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) from TensorflowHub.

## __Setup__

### _Install_

For tensorflow
```bash
  pip install -U "tensorflow-text==2.8.*"
  #pip install tf-models-official==2.7.0
  # Run into tensorflow_model module not found error. 
  # Try without specify version
  pip install tf-models-official
````

For running in vscode:
```bash
  conda install -n bert_finetune ipykernel --update-deps --force-reinstall
````

### _Import_

In [1]:
import os
import json
import pandas as pd
from pathlib import Path
from tqdm import tqdm

from sklearn import model_selection

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_models as tfm
from official.nlp import optimization

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

/home/shius/miniconda3/envs/bert_finetune/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### _Configuration info_

In [2]:
# Reproducibility
seed = 20220609

# Setting paths
work_dir          = Path.home() / "projects/plant_sci_hist/2_text_classify"
corpus_combo_file = work_dir / "corpus_combo"

# Dataset
batch_size     = 16
# Was 32, cannot run
# Changed to 16, run on RTX 3090
# Changed to 2, still cannot run on 3070.


shuffle_buffer = 2

# https://stackoverflow.com/questions/56613155/tensorflow-tf-data-autotune
# tf.data builds a performance model of the input pipeline and runs an 
# optimization algorithm to find a good allocation of its CPU budget across all
# parameters specified as AUTOTUNE
AUTOTUNE = tf.data.AUTOTUNE

# maximum number of tokens in a document
max_length = 512


## __Get text ready__

### _Read json to dataframe_

In [3]:
def split_train_validate_test(corpus_combo_file, rand_state):
  '''Load data and split train, validation, test subsets for the cleaned texts
  Args:
    corpus_combo_file (str): path to the json data file
    rand_state (int): for reproducibility
  Return:
    train, test, test (pandas dataframes): training, validation, testing sets
  '''
  # Load json file
  with corpus_combo_file.open("r+") as f:
      corpus_combo_json = json.load(f)

  # Convert json back to dataframe
  corpus_combo = pd.read_json(corpus_combo_json)

  # Cleaned corpus
  corpus = corpus_combo[['label','txt']]

  # Split train test
  train, test = model_selection.train_test_split(corpus, 
      test_size=0.2, stratify=corpus['label'], random_state=rand_state)

  # Split train validate
  train, valid = model_selection.train_test_split(train, 
      test_size=0.25, stratify=train['label'], random_state=rand_state)

  return train, valid, test

In [4]:
train, valid, test = split_train_validate_test(corpus_combo_file, seed)

### _Save text entries into files_

Follow the same structure as the IMDB dataset in the `aclImdb` folder.

In [5]:
corpus_dir = work_dir / "corpus_dir"

# Create train, valid, test dir
train_dir  = corpus_dir / 'train'
valid_dir  = corpus_dir / 'valid'
test_dir   = corpus_dir / 'test'

train_dir.mkdir(parents=True, exist_ok=True)
valid_dir.mkdir(parents=True, exist_ok=True)
test_dir.mkdir(parents=True, exist_ok=True)

In [6]:
# Create pos, neg directory for each
(train_dir / "pos").mkdir(parents=True, exist_ok=True)
(train_dir / "neg").mkdir(parents=True, exist_ok=True)
(valid_dir / "pos").mkdir(parents=True, exist_ok=True)
(valid_dir / "neg").mkdir(parents=True, exist_ok=True)
(test_dir  / "pos").mkdir(parents=True, exist_ok=True)
(test_dir  / "neg").mkdir(parents=True, exist_ok=True)

In [7]:
def save_entry_to_file(df, target_dir):
  '''Save each text entry in the dataframe as a file
  '''

  labels = df['label'].values
  txts   = df['txt'].values
  c_dict = {0:0, 1:0}
  for count, label in tqdm(enumerate(labels), total=len(labels)):
    if label == 0:
      c_dict[0] += 1
      with open(target_dir / f"neg/{count}.txt", "w") as f:
        f.write(txts[count])
    else:
      c_dict[1] += 1
      with open(target_dir / f"pos/{count}.txt", "w") as f:
        f.write(txts[count])

  print(c_dict)

In [8]:
# Make into comments because they have been done
#save_entry_to_file(train, train_dir)
#save_entry_to_file(valid, valid_dir)
#save_entry_to_file(test , test_dir)

### _Create datasets_

In [9]:
# Create train dataset
raw_train_ds = tf.keras.utils.text_dataset_from_directory(str(train_dir), 
                                                          batch_size=batch_size)
type(raw_train_ds)

Found 51987 files belonging to 2 classes.


2022-06-21 12:05:53.246457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-21 12:05:53.255299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-21 12:05:53.255708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-21 12:05:53.256389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [10]:
class_names = raw_train_ds.class_names
train_ds    = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
class_names, train_ds

(['neg', 'pos'],
 <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>)

In [11]:
# Get valition set
raw_valid_ds = tf.keras.utils.text_dataset_from_directory(str(valid_dir), 
                                                          batch_size=batch_size)
valid_ds     = raw_valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Get test set
raw_test_ds  = tf.keras.utils.text_dataset_from_directory(str(test_dir), 
                                                          batch_size=batch_size)
test_ds      = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 17329 files belonging to 2 classes.
Found 17330 files belonging to 2 classes.


In [12]:
# Check out one record
for text_batch, label_batch in train_ds.take(1):
  for i in range(1):
    print(f'Text: {text_batch.numpy()[i][:50]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

Text: b'Genome-Wide Identification of VQ Motif-Containing '
Label : 1 (pos)


2022-06-21 12:06:40.574739: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## __Define Hub models and initial testing__

### _Hub models to use_

Use BERT trained on MEDLINE/Pubmed:
- https://tfhub.dev/google/experts/bert/pubmed/2

In [13]:
tfhub_encoder = 'https://tfhub.dev/google/experts/bert/pubmed/2'
tfhub_preproc = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

### _Load and test preprocessing model_

In [14]:
bert_preprocess_model = hub.KerasLayer(tfhub_preproc)
type(bert_preprocess_model)

tensorflow_hub.keras_layer.KerasLayer

In [15]:
# Note this is not a layer but a saved model
bert_preprocess = hub.load(tfhub_preproc)
type(bert_preprocess)

tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject

In [16]:
text_test = ['This paper is about Plant, like maize, rice, and tomato!']

#######################
# CRITICAL STEP!!! NEED TO CHANGE DIMENSION From 128 to 512
#######################

tok = bert_preprocess.tokenize(tf.constant(text_test))
text_preprocessed = bert_preprocess.bert_pack_inputs([tok, tok], 
                                                     tf.constant(max_length))
#text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')

# The size is 128.
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :30]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :30]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :30]}')

Keys       : ['input_word_ids', 'input_type_ids', 'input_mask']
Shape      : (1, 512)
Word Ids   : [  101  2023  3259  2003  2055  3269  1010  2066 21154  1010  5785  1010
  1998 20856   999   102  2023  3259  2003  2055  3269  1010  2066 21154
  1010  5785  1010  1998 20856   999]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


### _Load and test BERT model_

In [17]:
bert_model = hub.KerasLayer(tfhub_encoder)
type(bert_model)

tensorflow_hub.keras_layer.KerasLayer

In [18]:
bert_results = bert_model(text_preprocessed)
print(f'Loaded BERT: {tfhub_encoder}')

Loaded BERT: https://tfhub.dev/google/experts/bert/pubmed/2


2022-06-21 12:06:54.636726: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [19]:
# pooled_output: embedding of the document
# 768: size of the embedding vector
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :7]}')

Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[ 0.19967358 -0.6023183   0.01133679 -0.94678575 -0.32777378  0.3923298
 -0.90228444]


In [20]:
# sequence_output: embeddings of each token
# 512: number of tokens of text_preprocessed
# 768: size of the embedding vector
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :2]}')

Sequence Outputs Shape:(1, 512, 768)
Sequence Outputs Values:[[ 0.20244306 -0.6965506   0.01134092 ...  0.07541777  0.804416
   0.43197766]
 [-0.8271446  -1.1451745   0.23592134 ... -0.15745929 -0.5484226
  -2.219939  ]]


In [21]:
# encoder_outputs: intermediate activation of a transformer block
# Q: Assuming activation is the output value of the activation function.
# 12: number of transformer blocks
print(f'Encoder Outputs length:{len(bert_results["encoder_outputs"])}')

# Saem as sequence output values
print(f'Sequence Outputs shape:{bert_results["encoder_outputs"][0].shape}')
print(f'Sequence Outputs shape:{bert_results["encoder_outputs"][0][0, :2]}')

Encoder Outputs length:12
Sequence Outputs shape:(1, 512, 768)
Sequence Outputs shape:[[ 0.04164825 -0.05509862  0.05931972 ... -0.07429437  0.01309458
   0.04005507]
 [-0.7039484  -0.5594542   0.16843846 ... -0.15212026  0.34699732
  -0.403786  ]]


## __Build classification model__

6/18/19
- The challenge is how to use bert_pack_inputs as a prprocessing layer.
- [This hub page](https://www.tensorflow.org/hub/common_saved_model_apis/text) has some info: does not help much, as the info is for individual instance and the model is very different from what I want.
- [Fine-tuning a BERT model](https://www.tensorflow.org/text/tutorials/fine_tune_bert) tutorial: Here the dataset is tokenized, packed, and them used as input to model. Try it and see.
- Also check the [preprocess model page](https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3) for syntax.

### _Get tokenizer to work_

In [22]:
# Original example
#tok = bert_preprocess.tokenize(tf.constant(text_test))
#text_preprocessed = bert_preprocess.bert_pack_inputs([tok, tok], 
#                                                     tf.constant(max_length))
tokenizer  = hub.KerasLayer(bert_preprocess.tokenize)

In [23]:
sentences1 = tf.constant(["hello tensorflow"])
sentences2 = tf.constant(["goodbye tensorflow"])
tokens1    = tokenizer(sentences1)
tokens2    = tokenizer(sentences2)
tokens1, tokens2

(<tf.RaggedTensor [[[7592], [23435, 12314]]]>,
 <tf.RaggedTensor [[[9119], [23435, 12314]]]>)

In [24]:
# The following throw an error:
# AttributeError: 'KerasLayer' object has no attribute 'get_special_tokens_dict'
#special = tokenizer.get_special_tokens_dict

# Ok, in the "Custom input packing and MLM support" section of this page:
# https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
# There is this line:
special = bert_preprocess.tokenize.get_special_tokens_dict()
special

{'padding_id': <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 'end_of_segment_id': <tf.Tensor: shape=(), dtype=int32, numpy=102>,
 'mask_id': <tf.Tensor: shape=(), dtype=int32, numpy=103>,
 'start_of_sequence_id': <tf.Tensor: shape=(), dtype=int32, numpy=101>,
 'vocab_size': <tf.Tensor: shape=(), dtype=int32, numpy=30522>}

### _Get Packer to work_

In [25]:
packer = tfm.nlp.layers.BertPackInputs(
    seq_length=max_length,
    special_tokens_dict = special)

In [26]:
packed = packer([tokens1, tokens2])
for key, tensor in packed.items():
  print(f"{key:15s}: {tensor.shape}, {tensor[:, :10]}")

input_word_ids : (1, 512), [[  101  7592 23435 12314   102  9119 23435 12314   102     0]]
input_mask     : (1, 512), [[1 1 1 1 1 1 1 1 1 0]]
input_type_ids : (1, 512), [[0 0 0 0 0 1 1 1 1 0]]


### _Combine tokenizer and packer into a layer_

The following class definition is from [this colab file](https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/fine_tune_bert.ipynb?hl=sv-SEIt&skip_cache=false).
- In the `train_ds.map(bert_inputs_processor).prefetch(1)` line below, there is this error:
  - TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__call() takes 2 positional arguments but 3 were given
- Looking into how the class is implemented, it is asking for `inputs['txt']` and `inputs['label']`. But train_ds does not have that kind of structure. This is likely the reason why this fails.
- So instead, put the tokenizer and packer into separate layers in the `define model architecture` section below.


In [27]:
# This does not work for me. 
class BertInputProcessor(tf.keras.layers.Layer):
  def __init__(self, tokenizer, packer):
    super().__init__()
    self.tokenizer = tokenizer
    self.packer = packer

  def call(self, inputs):
    # Original code is expecting two features, but I only have one.
    #tok1 = self.tokenizer(inputs['sentence1'])
    #tok2 = self.tokenizer(inputs['sentence2'])
    tok = self.tokenizer(inputs['txt'])

    packed = self.packer([tok, tok])

    if 'label' in inputs:
      return packed, inputs['label']
    else:
      return packed

In [28]:
#bert_inputs_processor = BertInputProcessor(tokenizer, packer)
#train_packed          = train_ds.map(bert_inputs_processor).prefetch(1)

### _Define model architecture_

In [29]:
def build_classifier_model():
  # Input layer
  text_input        = tf.keras.layers.Input(shape=(), dtype=tf.string, 
                                            name='txt')

  # Will this work??  
  tokenizer         = hub.KerasLayer(bert_preprocess.tokenize)
  special           = bert_preprocess.tokenize.get_special_tokens_dict()
  tokenizer_outputs = tokenizer(text_input)

  # Processing layer: This has the key change to allow longer texts.
  packer            = tfm.nlp.layers.BertPackInputs(
                      seq_length=max_length,
                      special_tokens_dict = special)

  #preproc_layer     = hub.KerasLayer(bert_preprocess.bert_pack_inputs, 
  #                                   arguments=dict(seq_lenght=max_length),
  #                                   name='preprocessing')
                                     
  packer_outputs   = packer(tokenizer_outputs)

  # Initialize encoder
  encoder           = hub.KerasLayer(tfhub_encoder, trainable=True, 
                                   name='BERT_encoder')
  encoder_outputs   = encoder(packer_outputs)
  
  # Q: Wonder if this is the dense layer mentioned above.
  print(type(encoder_outputs))

  # Get just the embeddings for each doc (ignore token level info)
  net            = encoder_outputs['pooled_output']

  # Dropout layer
  net            = tf.keras.layers.Dropout(0.1)(net)

  # output layer: single node, Q: Why??
  net            = tf.keras.layers.Dense(2, activation='softmax', 
                                                        name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [30]:
classifier_model = build_classifier_model()

<class 'dict'>
Raw result   : tf.Tensor([[0.54030627 0.4596937 ]], shape=(1, 2), dtype=float32)


In [ ]:
text_test        = ['Plant science focuses on studies of photosynthetic species.']

# tf.constant: create a Tensor from tensor like objects
tensor_test      = tf.constant(text_test)
bert_raw_result  = classifier_model(tensor_test)

print("Raw result   :", bert_raw_result)

In [31]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 txt (InputLayer)               [(None,)]            0           []                               
                                                                                                  
 keras_layer_3 (KerasLayer)     (None, None, None)   0           ['txt[0][0]']                    
                                                                                                  
 bert_pack_inputs_1 (BertPackIn  {'input_word_ids':   0          ['keras_layer_3[0][0]']          
 puts)                          (None, 512),                                                      
                                 'input_mask': (Non                                               
                                e, 512),                                                      

## __Model training__

### _Compile model_

In [32]:
epochs           = 20
cardinality      = tf.data.experimental.cardinality(train_ds)
steps_per_epoch  = cardinality.numpy()
num_train_steps  = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

# loss function: 
loss    = tf.keras.losses.SparseCategoricalCrossentropy()

# evaluation metrics
metrics = tf.metrics.BinaryAccuracy()

# Initial learning rate
init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [33]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

### _Train model_

In [34]:
# Specify callbacks

# early stopping
callback_es  = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# model checkpoint
cp_filepath  = work_dir / "model_ori_bert_tf_pubmed"
callback_mcp = tf.keras.callbacks.ModelCheckpoint(filepath=str(cp_filepath), 
              mode='max', save_weights_only=False, monitor='val_accuracy', 
              save_best_only=True)

In [35]:
print(f'Training model with {tfhub_encoder}')
history = classifier_model.fit(x=train_ds, 
                               batch_size=batch_size, 
                               epochs=epochs,
                               validation_data=valid_ds, 
                               verbose=1,
                               callbacks=[callback_es, callback_mcp])

Training model with https://tfhub.dev/google/experts/bert/pubmed/2
Epoch 1/20
3250/3250 [==============================] - ETA: 0s - loss: 0.1631 - binary_accuracy: 0.5000

KeyboardInterrupt: 

### Evaluate the model

Let's see how the model performs. Two values will be returned. Loss (a number which represents the error, lower values are better), and accuracy.

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

### Plot the accuracy and loss over time

Based on the `History` object returned by `model.fit()`. You can plot the training and validation loss for comparison, as well as the training and validation accuracy:

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In this plot, the red lines represent the training loss and accuracy, and the blue lines are the validation loss and accuracy.

## Export for inference

Now you just save your fine-tuned model for later use.

In [ ]:
dataset_name = 'imdb'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

classifier_model.save(saved_model_path, include_optimizer=False)

Let's reload the model, so you can try it side by side with the model that is still in memory.

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

Here you can test your model on any sentence you want, just add to the examples variable below.

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'this is such an amazing movie!',  # this is the same sentence tried earlier
    'The movie was great!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was terrible...'
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print('Results from the saved model:')
print_my_examples(examples, reloaded_results)
print('Results from the model in memory:')
print_my_examples(examples, original_results)

If you want to use your model on [TF Serving](https://www.tensorflow.org/tfx/guide/serving), remember that it will call your SavedModel through one of its named signatures. In Python, you can test them as follows:

In [ ]:
serving_results = reloaded_model \
            .signatures['serving_default'](tf.constant(examples))

serving_results = tf.sigmoid(serving_results['classifier'])

print_my_examples(examples, serving_results)

# __FAILED STEPS__


## _[Gave up on this] Convert training dataframe to dataset_

Try many different ways, was trying not to slit text records into different files then load as a dataset like the tutorial. But converting from dataframe always missed something that I cannot quite put my finger on. Moving on. Revisit this later...

- See the [pd_dataframe_to_tf_dataset](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/pd_dataframe_to_tf_dataset) function, but this needs tf 2.9, conflict with tensorflow_text.
- See [this](https://www.tensorflow.org/tutorials/load_data/pandas_dataframe): See the shuffle and batch functions. Does not work...
- See [this post](https://medium.com/when-i-work-data/converting-a-pandas-dataframe-into-a-tensorflow-dataset-752f3783c168):  Was able to create SicedDataset, then BatchDatabase after applying the batch function, then PrefetchDataset. But trying to retreive a test example from trainin dataset lead to:
  - InvalidArgumentError: Index out of range using input dim 0; input has only 0 dims [Op:StridedSlice] name: strided_slice/
  - Ok, as I was implmenting the next solution, realize that I did not call the right obj for prefetch. Can be the reason why.
- Ah, see [this post](https://stackoverflow.com/questions/58461609/how-to-convert-pandas-dataframe-to-tensorflow-dataset): key is to turn train_data to dictionary before calling from_tensor_slices.
  - A little comment below say need to do .to_dict() instead which make sense. Because if just do dict(train), the thing finish in 0.1 sec which does not make sense. But this fails and throw:
    - ValueError: Unbatching a tensor is only supported for rank >= 1
  - Found [this post](https://stackoverflow.com/questions/55560620/valueerror-unbatching-a-tensor-is-only-supported-for-rank-1): Now try to uses this syntax. Still does not work...

#### The following DOES NOT work

```Python
raw_train_ds = (tf.data.Dataset.from_tensor_slices(
        (tf.cast(train['txt'].values, tf.string),
         tf.cast(train['label'].values, tf.int32),)))
raw_train_ds = tf.data.Dataset.from_tensor_slices(train)
raw_train_ds = tf.data.Dataset.from_tensor_slices(dict(train))
raw_train_ds = tf.data.Dataset.from_tensor_slices(train.to_dict())
raw_train_ds = tf.data.Dataset.from_tensor_slices((y_train,X_train))
raw_train_ds = tf.data.Dataset.from_tensor_slices((i_train, 
                                                   y_train.values,
                                                   X_train.values))

The last one is almost working, but at later stage when I try to pack the input, it breaks. Upon closer examination, the train_ds looks like:

```
<PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
```

But what I need is more like:

```
<PrefetchDataset element_spec={'idx': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'sentence1': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'sentence2': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>
```

In [ ]:
trial = train.iloc[:3,:]
raw_trial_ds = (tf.data.Dataset.from_tensor_slices(
        (tf.cast(trial['txt'].values, tf.string),
         tf.cast(trial['label'].values, tf.int32),)))
raw_trial_ds
# Does not work

In [ ]:
# Call "trial" so it does not mix with the testing set
trial_dict   = {"idx":[123,322], "label":[0,1], 
               "txt":["The 1st sentence", "The second"]}

# This creates a TensorSliceDataset
raw_trial_ds =  tf.data.Dataset.from_tensor_slices(trial_dict)

# Now this has the right structure!!!
raw_trial_ds

In [ ]:
for index_batch, label_batch, text_batch in raw_trial_ds.take(1):
  for i in range(1):
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')
    print(f'Review: {text_batch.numpy()[i]}')


In [ ]:
raw_trial_ds_batch = raw_trial_ds.batch(batch_size)
type(raw_trial_ds_batch)

In [ ]:
trial_ds = raw_trial_ds_batch.cache().prefetch(buffer_size=AUTOTUNE)
len(trial_ds)

In [ ]:
# Still have the right structure.
trial_ds.element_spec['txt']

In [ ]:
take1 = trial_ds.take(1)
help(take1)
    

#### Convert train, valid, and test sets

In [ ]:
def dataframe_to_dataset(df):
  '''Convert DataFrame to PrefetchDataset
  Args:
    df (dataframe): with two columns ('txt', and 'label') and indices
  Return:
    dataset (PrefetchDataset): with:
      <PrefetchDataset element_spec={'idx': TensorSpec(shape=(None,), 
                                            dtype=tf.int32, name=None), 
                                     'label': TensorSpec(shape=(None,), 
                                            dtype=tf.int32, name=None), 
                                     'txt': TensorSpec(shape=(None,), 
                                            dtype=tf.string, name=None)}>

  '''
  idx = df.index   # aded this, as the tutorial has an index input
  X   = df['txt']
  y   = df['label']

  df_dict = {"idx":idx, "label":y.values, "txt":X.values}

  raw_ds       = tf.data.Dataset.from_tensor_slices(df_dict)  
  raw_ds_batch = raw_ds.batch(batch_size)
  dataset      = raw_ds_batch.cache().prefetch(buffer_size=AUTOTUNE)
  
  return dataset

In [ ]:
# Convert train data
train_ds = dataframe_to_dataset(train)
train_ds

In [ ]:
# Get validation dataset
valid_ds = dataframe_to_dataset(valid)

# Get testing dataset
test_ds = dataframe_to_dataset(test)

### _Testing_

In [ ]:
train_ds.take(1)

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  print(text_batch[0])
  print(len(text_batch))
  print(label_batch)
  print(len(label_batch))

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(1):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')